In [27]:
import time
import collections
import numpy as np
import pandas as pd
import xlwings as xw
from difflib import SequenceMatcher

In [28]:
# 定义相似度匹配函数
def similarity_match(a, b, threshold):
    similarity = SequenceMatcher(None, a, b).ratio()
    return similarity >= threshold

In [29]:
# 
def similarity_row(r1, r2):
    if r1[2] != r2[2]:
        return False
    if r1[1] != r2[1]:
        return False
    if r1[0] != r2[0]:
        return False
    # 客户名称
    return similarity_match(r1[3], r2[3], 0.8)

In [30]:
path = r"C:\Users\Administrator\Desktop\NewFolder\data\small.xlsx"
cmi_name = "CMI原始流向"
business_name = "商业收集原始流向"
cmi_column = ["销售时间", "产品规格", "原始数量", "原始客户名称"]
business_column = ["销售日期", "匹配的标准名称", "数量", "客户名称"]
compare_column = ["销售时间", "产品规格", "数量", "客户名称"]
st = time.time()


In [31]:
app = xw.App(visible=False, add_book=False)
app.display_alerts = False
app.screen_updating = False
wb = app.books.open(path)

In [32]:
sheet_cmi = wb.sheets[cmi_name]
df_cmi = pd.DataFrame(columns=compare_column)
header_row = sheet_cmi.range('A1').expand('right').value
for df_name, name in zip(compare_column, cmi_column):
    index = header_row.index(name)
    each_data = sheet_cmi.range(2, index+1).expand('down').value
    df_cmi[df_name] = each_data
print(f"完成{cmi_name}的数据读取")

完成CMI原始流向的数据读取


In [33]:
sheet_business = wb.sheets[business_name]
df_business = pd.DataFrame(columns=compare_column)
header_row = sheet_business.range('A1').expand('right').value
for df_name, name in zip(compare_column, business_column):
    index = header_row.index(name)
    each_data = sheet_business.range(2, index+1).expand('down').value
    df_business[df_name] = each_data
print(f"完成{business_name}的数据读取")

完成商业收集原始流向的数据读取


In [34]:
diff = pd.concat([df_cmi, df_business])
# 进行日期格式化操作
diff['销售时间']=pd.to_datetime(diff['销售时间'])
diff['销售时间'] = diff['销售时间'].dt.date
diff = diff.reindex(columns=compare_column)
diff['来源'] = [cmi_name] * len(df_cmi) + [business_name] * len(df_business)
diff.sort_values(by=[*compare_column, "来源"], inplace=True)
print("开始数据差异比对")

开始数据差异比对


In [35]:
# 去除重复选项
diff_cmi = collections.deque()
diff_business = collections.deque()
for _, row in diff.iterrows():
    if row["来源"] == cmi_name:
        if len(diff_business) == 0:
            diff_cmi.append(row)
            continue
        business_row = diff_business.pop()
        if similarity_row(row, business_row):
            continue
        diff_cmi.append(row)
        diff_business.append(business_row)
    else:
        if len(diff_cmi) == 0:
            diff_business.append(row)
            continue
        cmi_row = diff_cmi.pop()
        if similarity_row(cmi_row, row):
            continue
        diff_cmi.append(cmi_row)
        diff_business.append(row)
print("数据差异比对结束，结果如下:")
print(f"{cmi_name}差异数量:{len(diff_cmi)}")
print(f"{business_name}差异数量:{len(diff_business)}")

数据差异比对结束，结果如下:
CMI原始流向差异数量:359
商业收集原始流向差异数量:1772


In [36]:
diff_cmi = pd.DataFrame(diff_cmi)
diff_business = pd.DataFrame(diff_business)
diff_cmi_index = np.array(diff_cmi.index) + 2
diff_business_index = np.array(diff_business.index) + 2

In [37]:
print("正在进行标红处理")
for sheet, diff_index in [[sheet_cmi, diff_cmi_index], [sheet_business, diff_business_index]]:
    for index in diff_index:
        sheet.range((index, 1)).color = (255, 0, 0)
wb.save(path)
print("标红处理已结束")


正在进行标红处理
标红处理已结束


In [38]:
wb.close()
app.quit()

In [39]:
print(time.time() - st)

24.88104486465454


In [47]:
is_check = False
if is_check:
    diff_check = pd.concat([diff_cmi, diff_business])
    diff_check.sort_values(by=[*compare_column, "来源"], inplace=True)
    